# Title

In [201]:
import numpy as np
import pandas as pd
import nltk
nltk.download('stopwords')
nltk.download('words')
import os
import re
import sqlalchemy

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\brand\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\brand\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


## Dataset

In [202]:
CWD = os.getcwd()
dataset_engine = sqlalchemy.create_engine(f"sqlite:///{CWD}/modeling_dataset.db")
dataset_engine.connect()

In [203]:
tables = pd.read_sql("SELECT * FROM sqlite_master WHERE type = 'table'", dataset_engine)
tables

,type,name,tbl_name,rootpage,sql
0,table,by,by,2,"CREATE TABLE ""by"" (\n\t""index"" BIGINT, \n\tlic..."
1,table,by-sa,by-sa,28,"CREATE TABLE ""by-sa"" (\n\t""index"" BIGINT, \n\t..."
2,table,by-nc,by-nc,410,"CREATE TABLE ""by-nc"" (\n\t""index"" BIGINT, \n\t..."
3,table,by-nc-sa,by-nc-sa,436,"CREATE TABLE ""by-nc-sa"" (\n\t""index"" BIGINT, \..."
4,table,by-nd,by-nd,1223,"CREATE TABLE ""by-nd"" (\n\t""index"" BIGINT, \n\t..."
5,table,by-nc-nd,by-nc-nd,1590,"CREATE TABLE ""by-nc-nd"" (\n\t""index"" BIGINT, \..."
6,table,publicdomain,publicdomain,803,"CREATE TABLE publicdomain (\n\t""index"" BIGINT,..."


In [204]:
webpages_dataset = pd.concat(
    [
        pd.read_sql(f"SELECT * FROM '{table_name}'", dataset_engine)
        for table_name in tables["name"]
    ]
)
webpages_dataset = webpages_dataset\
    .loc[webpages_dataset["contents"] != "", :]\
    .reset_index()\
    .drop(["index", "title", "level_0"], axis=1)
webpages_dataset.sample(5)

,license,url,contents
8162,licenses/by-nc-nd/3.0,https://learning.hccs.edu/faculty/elisa.diehl/...,Sound Credits for The Spoilers 2022 — HCC Lear...
2320,licenses/by-sa/2.0,https://github.com/beatrizmilz/blog-en/blob/ma...,blog-en/_quarto.yml at main · beatrizmilz/blog...
6016,licenses/by-nd/2.5,https://www.lawfareblog.com/justice-department...,The Justice Department Indicted Russian Nation...
4220,licenses/by-nc-sa/2.5,https://www.mediawiki.org/wiki/Module:Extensio...,Module:Extension/sandbox - MediaWiki Jump to c...
7404,licenses/by-nd-nc/2.0,https://libguides.colorado.edu/c.php?g=887144&...,CC Licenses - Creative Commons - Research Guid...


In [205]:
webpages_dataset_deduplicate = webpages_dataset.groupby("url").first()\
    .reset_index()
webpages_dataset_deduplicate.describe()

,url,license,contents
count,3238,3238,3238
unique,3238,39,3123
top,http://abcr-mefmo.org/index.php/abcr/about,licenses/by/2.1,403 Forbidden 403 Forbidden nginx
freq,1,184,44


In [206]:
import dataset_sampling
license_map = dataset_sampling.get_license_map()
license_ser = pd.concat([v for v in license_map.values()])
license_ser_splits_df = license_ser.str.split("/", expand=True)
license_ser_splits_df = license_ser_splits_df.rename(
    columns = {
        0: "Tool Typing",
        1: "General Typing",
        2: "Version",
        3: "Jurisdiction"
    }
)
license_ser_splits_df["General Typing"] = license_ser_splits_df["General Typing"].str.replace("mark|zero", "publicdomain", regex=True)
license_ser_splits_df["General Typing"] = license_ser_splits_df["General Typing"].str.replace("by-nd-nc", "by-nc-nd", regex=True)
license_ser_splits_df["Version"] = license_ser_splits_df["Version"].astype(float)
license_one_hot_encoding = pd.DataFrame()
license_one_hot_encoding["by"] = license_ser_splits_df["General Typing"].str.contains("by")
license_one_hot_encoding["sa"] = license_ser_splits_df["General Typing"].str.contains("sa")
license_one_hot_encoding["nc"] = license_ser_splits_df["General Typing"].str.contains("nc")
license_one_hot_encoding["nd"] = license_ser_splits_df["General Typing"].str.contains("nd")
license_not_six_type = license_ser_splits_df["General Typing"].str.contains("by|sa|nc|nd")
license_one_hot_encoding["neither"] = ~(license_not_six_type.fillna(False))
license_df = pd.concat([license_ser, license_ser_splits_df, license_one_hot_encoding], axis = 1)\
    .rename(columns = {0: "license"})
license_df.head(6)

,license,Tool Typing,General Typing,Version,by,sa,nc,nd,neither
0,licenses/by/1.0,licenses,by,1.0,True,False,False,False,False
14,licenses/by/2.0,licenses,by,2.0,True,False,False,False,False
27,licenses/by/2.1,licenses,by,2.1,True,False,False,False,False
33,licenses/by/2.5,licenses,by,2.5,True,False,False,False,False
39,licenses/by/3.0,licenses,by,3.0,True,False,False,False,False
45,licenses/by/4.0,licenses,by,4.0,True,False,False,False,False


In [207]:
webpages_dataset_deduplicate = webpages_dataset_deduplicate.merge(license_df, on = "license")
webpages_dataset_deduplicate.sample(5)

,url,license,contents,Tool Typing,General Typing,Version,by,sa,nc,nd,neither
1836,https://www.leidensecurityandglobalaffairs.nl/...,licenses/by-nc/2.0,Emotional Labor During Disruptive Times: a Col...,licenses,by-nc,2.0,True,False,True,False,False
2799,https://www.healthcaredive.com/news/soc-teleme...,licenses/by-sa/2.0,SOC Telemed to be acquired by Healthcare Merge...,licenses,by-sa,2.0,True,True,False,False,False
2970,https://scholar.dominican.edu/all-faculty/170/,licenses/by-nd/3.0,"""Big History, Big Lesson"" by Mojgan Behmand an...",licenses,by-nd,3.0,True,False,False,True,False
2239,https://blog.to.com/advisory-webacms-2-1-0-cro...,licenses/by/2.1,Advisory: WebACMS 2.1.0 Cross-Site Scripting -...,licenses,by,2.1,True,False,False,False,False
2794,https://www.esa.int/Services/Creative_Commons_...,licenses/by-sa/2.0,ESA - Creative Commons Attribution-ShareAlike ...,licenses,by-sa,2.0,True,True,False,False,False


In [208]:
webpages_dataset_deduplicate.groupby("General Typing").count()

,url,license,contents,Tool Typing,Version,by,sa,nc,nd,neither
General Typing,,,,,,,,,,
by,985,985,985,985,985,985,985,985,985,985
by-nc,530,530,530,530,530,530,530,530,530,530
by-nc-nd,208,208,208,208,208,208,208,208,208,208
by-nc-sa,360,360,360,360,360,360,360,360,360,360
by-nd,318,318,318,318,318,318,318,318,318,318
by-sa,588,588,588,588,588,588,588,588,588,588
publicdomain,249,249,249,249,249,249,249,249,249,249


In [209]:
webpages_dataset_deduplicate["contents"] = webpages_dataset_deduplicate["contents"].apply(lambda x: x[:4000])

In [210]:
import string
def remove_unicodes(ser):
    return ser.map(lambda x: " ".join([c for c in x if c in string.printable]))

def has_unicodes(s, tolerance = 20):
    return np.sum([c not in string.printable for c in s]) <= tolerance

def not_well_decrypted(s, tolerance = 25):
    words = re.split(r"\s+", s)
    return np.sum([len(c) == 1 for c in words]) <= tolerance

def remove_unicodes_aggressive(df, field_name = "contents"):
    df_remove_unicode = df.loc[df[field_name].apply(has_unicodes), :]
    df_remove_unicode = df_remove_unicode.loc[
        df_remove_unicode[field_name].str.len() >= 600, :
    ]
    return df_remove_unicode


In [211]:
# Less Aggressive pruning
webpages_dataset_deduplicate["parsed_contents"] = remove_unicodes(webpages_dataset_deduplicate["contents"])
#webpages_dataset_deduplicate.loc[1149, ["contents", "parsed_contents"]]

# More Aggressive pruning
#webpages_dataset_deduplicate = remove_unicodes_aggressive(webpages_dataset_deduplicate)

In [212]:
def remove_less_than_c_chars(s, tolerance = 1):
    words = re.split(r"\s+", s)
    return " ".join([c for c in words if len(c) > tolerance and "obj" not in c])

def remove_more_than_c_chars(s, tolerance = 15):
    words = re.split(r"\s+", s)
    return " ".join([c for c in words if len(c) <= tolerance])

def remove_non_english(s):
    words = set(nltk.corpus.words.words())
    return " ".join(w for w in re.split(r"\s+", s) if w in words)

def remove_web_urls(s):
    return re.sub(r"[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)", " ", s)

def clear_stopwords(s):
    return " ".join([word for word in re.split(r"\s+", s) if word.lower() not in nltk.corpus.stopwords.words('english')])

def clear_stopwords_series(ser):
    return ser.apply(clear_stopwords)

def overall_cleaning(ser):
    cleaned_ser = ser.str.lower()
    cleaned_ser = cleaned_ser.apply(remove_web_urls)
    cleaned_ser = cleaned_ser.str.replace(r'[^A-Za-z\s]', ' ', regex = True)
    cleaned_ser = cleaned_ser.str.replace(r"\s+", " ", regex = True)
    cleaned_ser = cleaned_ser.apply(remove_less_than_c_chars)
    cleaned_ser = cleaned_ser.apply(remove_non_english)
    #cleaned_ser = cleaned_ser.apply(remove_more_than_c_chars)
    return cleaned_ser

webpages_dataset_deduplicate["cleaned_contents"] = overall_cleaning(webpages_dataset_deduplicate["contents"])
#webpages_dataset_deduplicate["parsed_cleaned_contents"] = clear_stopwords_series(webpages_dataset_deduplicate["cleaned_contents"])

In [213]:
for row in webpages_dataset_deduplicate["cleaned_contents"].sample(5):
    print(
        f"Entry content:\n{row[:700]}\n"
    )

Entry content:
just moment if the site connection is secure enable and to continue needs to review the security of your connection before proceeding ray id performance security by

Entry content:
profile ma ce oak ra ta ea do en

Entry content:
therapeutic resistance in early rheumatoid arthritis epigenetic annals of the rheumatic skip to main content subscribe log in more log in via institution log in via log in your and password for personal or of institutional password forgot your log in register new account forgot your user name or password basket search more search for this advanced search latest content current issue archive about search for this advanced search close more main menu latest content current issue archive about subscribe log in more log in via institution log in via log in your and password for personal or of institutional password forgot your log in register new account forgot your user name or password you are

Entry content:
investigative magnetic resonance about

## Preprocessing

In [214]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import top_k_accuracy_score
from sklearn.decomposition import PCA
import re

In [215]:
def tokenize_url(url):
    return " ".join(re.split(r"[_/\.-]", re.sub(r"\d", "", url)))
tokenize_url("creativecommons.org/licenses/by-sa/4.0")

'creativecommons org licenses by sa  '

In [216]:
dataset = webpages_dataset_deduplicate.copy()
dataset['token_url'] = dataset["url"].apply(tokenize_url)
dataset["train_text"] = dataset["token_url"] + " " + dataset["cleaned_contents"]

In [217]:
def extract_text_features_tfidf(train, test, text_field = "train_text"):
    tfidf_vectorizer = TfidfVectorizer(use_idf=True, max_df=0.9, stop_words="english")
    tfidf_vectorizer.fit_transform(train[text_field].values)
    train_vectorized = tfidf_vectorizer.transform(train[text_field].values)
    test_vectorized = tfidf_vectorizer.transform(test[text_field].values)
    return train_vectorized, test_vectorized, tfidf_vectorizer

## Modeling

In [218]:
from imblearn.over_sampling import SMOTE

In [219]:
dataset.groupby("General Typing")[["url"]].count().reset_index()["url"]

0    985
1    530
2    208
3    360
4    318
5    588
6    249
Name: url, dtype: int64

In [232]:
model_dataset = pd.DataFrame()
dataset_counts = dataset.groupby("General Typing")[["url"]].count().reset_index()["url"]
license_dict = {
        "by": 0,
        "by-nc": 1,
        "by-nc-nd": 2,
        "by-nc-sa": 3,
        "by-nd": 4,
        "by-sa": 5,
        "publicdomain": 6
}

model_dataset["train_text"], model_dataset["General Typing"] = dataset["cleaned_contents"], dataset["General Typing"]
model_dataset["General Typing"].replace(
    license_dict,
    inplace = True
)

training_set, test_set = train_test_split(model_dataset, test_size = 0.5)
Y_train = training_set["General Typing"].values
Y_test = test_set["General Typing"].values
X_train, X_test, model_vecter = extract_text_features_tfidf(
    training_set, test_set
)
'''
smote = SMOTE(
    sampling_strategy={k: max(int(max(dataset_counts) * 0.6), int(dataset_counts.iloc[k] * 1.4)) for k in range(7)} #
)
X_train, Y_train = smote.fit_resample(X_train, Y_train)
'''

'\nsmote = SMOTE(\n    sampling_strategy={k: max(int(max(dataset_counts) * 0.6), int(dataset_counts.iloc[k] * 1.4)) for k in range(7)} #\n)\nX_train, Y_train = smote.fit_resample(X_train, Y_train)\n'

### log reg

In [235]:
model_logreg = LogisticRegression(
    verbose = 0,
    max_iter = 1000,
    penalty = 'l2',
    solver = "liblinear",
    C = 5
).fit(X_train, Y_train)

In [236]:
print(
    top_k_accuracy_score(Y_train, model_logreg.predict_proba(X_train), k = 1),
    "\n",
    top_k_accuracy_score(Y_train, model_logreg.predict_proba(X_train), k = 2),
    "\n",
    top_k_accuracy_score(Y_train, model_logreg.predict_proba(X_train), k = 3),
    "\n==============\n",
    top_k_accuracy_score(Y_test, model_logreg.predict_proba(X_test), k = 1),
    "\n",
    top_k_accuracy_score(Y_test, model_logreg.predict_proba(X_test), k = 2),
    "\n",
    top_k_accuracy_score(Y_test, model_logreg.predict_proba(X_test), k = 3)
)

0.9252625077208153 
 0.9746757257566399 
 0.9870290302655961 
 0.38604076590487957 
 0.5663990117356393 
 0.7022853613341569


### SVM

In [237]:
model_svc = SVC(
    C = 5,
    verbose = 1,
    probability = True
).fit(X_train, Y_train)

[LibSVM]

In [238]:
print(
    top_k_accuracy_score(Y_train, model_svc.predict_proba(X_train), k = 1),
    "\n",
    top_k_accuracy_score(Y_train, model_svc.predict_proba(X_train), k = 2),
    "\n",
    top_k_accuracy_score(Y_train, model_svc.predict_proba(X_train), k = 3),
    "\n==============\n",
    top_k_accuracy_score(Y_test, model_svc.predict_proba(X_test), k = 1),
    "\n",
    top_k_accuracy_score(Y_test, model_svc.predict_proba(X_test), k = 2),
    "\n",
    top_k_accuracy_score(Y_test, model_svc.predict_proba(X_test), k = 3)
)

0.9376158122297714 
 0.9765287214329833 
 0.9888820259419395 
 0.40271772699197034 
 0.58307597282273 
 0.6991970352069179


### Tensorflow

In [121]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

In [122]:
tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2'
tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)
bert_model = hub.KerasLayer(tfhub_handle_encoder)

In [128]:
AUTOTUNE = tf.data.AUTOTUNE
BATCH_SIZE = 32
TRAINING_RATIO = 0.7

model_dataset_copy = dataset.loc[:, ["cleaned_contents", "General Typing"]]
model_dataset_copy["General Typing"].replace(
    license_dict,
    inplace = True
)
target = model_dataset_copy.pop("General Typing")
# dataset_tensor = tf.convert_to_tensor(model_dataset, dtype=np.string_)
tf_dataset = (
    tf.data.Dataset
    .from_tensor_slices((model_dataset_copy, target))
    .shuffle(50)
)
train_dataset = tf_dataset.take(int(TRAINING_RATIO * len(tf_dataset)))
test_dataset = tf_dataset.skip(int(TRAINING_RATIO * len(tf_dataset)))
val_dataset = test_dataset.skip(
    int((1 - TRAINING_RATIO) * 0.5 * len(tf_dataset))
)
test_dataset = test_dataset.take(
    int((1 - TRAINING_RATIO) * 0.5 * len(tf_dataset))
)

train_dataset = train_dataset.batch(BATCH_SIZE)
val_dataset = val_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [129]:
def build_classifier_model():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='parsed_cleaned_contents')
    preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']
    net = tf.keras.layers.Dropout(0.5)(net)
    net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
    return tf.keras.Model(text_input, net)

In [130]:
from tensorflow_addons.metrics import F1Score

In [131]:
epochs = 10
steps_per_epoch = tf.data.experimental.cardinality(train_dataset).numpy()
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 1e-7
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

In [132]:
classifier_model = build_classifier_model()
classifier_model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=["accuracy"]
)
history = classifier_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=epochs
)

Epoch 1/10
31/31 [==============================] - 110s 3s/step - loss: 2.7991e-07 - accuracy: 0.2700 - val_loss: 4.2688e-07 - val_accuracy: 0.0429
Epoch 2/10
31/31 [==============================] - 109s 4s/step - loss: 2.7697e-07 - accuracy: 0.2423 - val_loss: 4.3029e-07 - val_accuracy: 0.0286
Epoch 3/10
17/31 [===============>..............] - ETA: 45s - loss: 1.8846e-07 - accuracy: 0.3033

KeyboardInterrupt: 